In [38]:
import numpy as np
import pandas as pd
import re,math
import concurrent.futures
import multiprocessing
import threading
# import logging
import time


# 高相似多少
coum=100
# 加载保存的数组
# lin = np.load("lin.npy")
# lin = np.linspace(0,3,401)
lin = np.linspace(1,10,901)

# 设置日志级别
# logging.basicConfig(level=logging.DEBUG)

# 设置函数运行计时用的修饰器
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds')
        return result
    return wrapper


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx')
cells_name = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx', header=0).columns[2:] # 保证cells的名字从第一行的第三个开始
drug_names = list(data1.index)

# 合并drugA和drugB为一列作为索引
# 顺序
temp1 = [] # 分子指纹合并值 【【】，【】，【】，】
index_1 = [] # 对应的名字列表 【【】，【】，【】，】
# 逆序
temp2 = []
index_2 = []
for index,row in data2.iterrows():
    ct = list((np.array(data1.loc[row["DrugA"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugB"],:]))))
    temp1.append(ct)
    index_1.append((row["DrugA"],row["DrugB"]))
    ct = list((np.array(data1.loc[row["DrugB"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugA"],:]))))
    temp2.append(ct)
    index_2.append((row["DrugB"],row["DrugA"]))

/root/miniconda3/envs/jupyter-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [39]:
lin

array([ 1.  ,  1.01,  1.02,  1.03,  1.04,  1.05,  1.06,  1.07,  1.08,
        1.09,  1.1 ,  1.11,  1.12,  1.13,  1.14,  1.15,  1.16,  1.17,
        1.18,  1.19,  1.2 ,  1.21,  1.22,  1.23,  1.24,  1.25,  1.26,
        1.27,  1.28,  1.29,  1.3 ,  1.31,  1.32,  1.33,  1.34,  1.35,
        1.36,  1.37,  1.38,  1.39,  1.4 ,  1.41,  1.42,  1.43,  1.44,
        1.45,  1.46,  1.47,  1.48,  1.49,  1.5 ,  1.51,  1.52,  1.53,
        1.54,  1.55,  1.56,  1.57,  1.58,  1.59,  1.6 ,  1.61,  1.62,
        1.63,  1.64,  1.65,  1.66,  1.67,  1.68,  1.69,  1.7 ,  1.71,
        1.72,  1.73,  1.74,  1.75,  1.76,  1.77,  1.78,  1.79,  1.8 ,
        1.81,  1.82,  1.83,  1.84,  1.85,  1.86,  1.87,  1.88,  1.89,
        1.9 ,  1.91,  1.92,  1.93,  1.94,  1.95,  1.96,  1.97,  1.98,
        1.99,  2.  ,  2.01,  2.02,  2.03,  2.04,  2.05,  2.06,  2.07,
        2.08,  2.09,  2.1 ,  2.11,  2.12,  2.13,  2.14,  2.15,  2.16,
        2.17,  2.18,  2.19,  2.2 ,  2.21,  2.22,  2.23,  2.24,  2.25,
        2.26,  2.27,

In [40]:
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]
        b=q[i]
        if (a==1)|(b==1): # 并
            tep1=tep1+1
        if (a==1)&(b==1): # 交
            tep2=tep2+1
    c=round((tep2 / tep1),4)      #取值4位数
    return c

def compute_tanimoto(i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx):
    tmp=[]
    a = np.array(i) #取出一分子指纹 转换列表为向量
    b = np.array(j)
    for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
        c = np.array(k)
        d = np.array(m)
        tp1 = tanimoto(a,c)
        tp2 = tanimoto(a,d)
        tp3 = tanimoto(b,c)
        tp4 = tanimoto(b,d)
        tps = [tp1, tp2, tp3, tp4]
        index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
        index_location = tps.index(max(tps))
        tmp.append(max(tps))
        Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
    return idx, tmp

temp = [None] * len(temp1)
Tanimoto_index_dist = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 提交任务时使用 enumerate ，并将索引值作为参数传递给任务函数
    futures = [executor.submit(compute_tanimoto, i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx)
               for idx, (i,j,i_n,j_n) in enumerate(zip(temp1,temp2,index_1,index_2))]
    for future in concurrent.futures.as_completed(futures):
        idx, result = future.result()
        temp[idx] = result
Tanimoto = pd.DataFrame(temp, columns=index_1, index=index_1)

In [41]:
# 获取columns数组 - 处理后的
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1]) # 去除首尾括号的药物对名字
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp)) # 药物对协同得分
columns = data2.columns # 修改后的药物对名字

In [42]:
# 删去对应行后的字典
dict_lines = {} # {键1：值1} 谷本系数字典
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct]) # 每一列谷本系数 [:,1] 第一列的每一行 实际就是第一列
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y # 谷本系数药物对的名字做键1，y是值1
    ct += 1

In [43]:
# 分离x y数据构造字典
dict_columns_split = {} # 高相似字典
# 计算x数据
x = data2.drop(columns=columns)  # 其他所有的列
for i in columns:
    # b 要预测协同得分那一列
    y = data2.loc[:, i]
    # A 权重
    x_i = x.copy()
    x_i['y'] = y

    # 高相似的    取高相似的药物对名字max1和对应的值max5
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index = tp)
    df.sort_values(by=0, inplace=True, ascending=True)
    max_l = df.index[-coum:]
    max_5x = pd.DataFrame([data2.loc[:, j] for j in max_l]).T

    # 存储 对应数据
    count = 0 # 第几行 int
    for j in list(y.index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
        dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
        dict_columns_split[f'{i},{j}']['y_line']=y[count]
        dict_columns_split[f'{i},{j}']['max_x_line']=max_5x.iloc[count,:].values
        count += 1

In [44]:
TTT = [] # 值列表
cct = 0
NNN = []
for i in Tanimoto.index: 
    y = list(Tanimoto.iloc[:,cct])
    y.pop(cct) 
    tp_columns = list(columns.copy())
    tp_columns.pop(cct)
    df = pd.DataFrame(y, index=tp_columns)
    df.sort_values(by=0 , inplace=True, ascending=True) 
    max_l = df.index[-coum:]
    NNN.extend(max_l)
    # 计算max_5y
    max_5y = []
    for j in max_l:
        max_5y.append(Tanimoto.iloc[list(columns).index(j), cct])
    # 将max_5y加入TTT
    TTT += max_5y
    # 修改cct变量的值
    cct += 1

In [45]:
def alpha_pcc(t_coum, a, lin, linename, dict_columns_split, TTT, coum, data2, NNN):
    alpha = []
    result_pcc = []
    pcc_split = []
    pre_dict = []
    ori_dict = []
    for alp in lin:
        pre = []
        for elem in range(39):
            pp = [] 
            if TTT[(coum * a - t_coum):(coum * a)] == []:
                print(1111111111111)
            for j in TTT[(coum * a - t_coum):(coum * a)]: 
                pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
            pp = np.array(pp)  # 将 pp 转换为 numpy 数组
            pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
        pre = np.array(pre)  # 将 pre 转换为 numpy 数组
        pre_dict.append(pre)
        ori_dict.append(data2[linename])
        pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
    result_pcc.append(np.max(pcc_split))
    alpha.append(lin[pcc_split.index(np.max(pcc_split))])
    pre = pre_dict[pcc_split.index(np.max(pcc_split))]
    ori = ori_dict[pcc_split.index(np.max(pcc_split))]
    pcc=result_pcc[0]
    temp_NNN = NNN[(coum * a - t_coum):(coum * a)]
    temp_TTT = TTT[(coum * a - t_coum):(coum * a)]
    del result_pcc, pcc_split, pp
    return alpha, pcc, temp_NNN, temp_TTT, pre, ori

def alp_mse_rmse(alpha, t_coum, a, linename, dict_columns_split, TTT, coum):
    r1 = []  # 初始化 r1 为空列表
    alp = alpha[0]
    for elem in range(39):
        pp = []
        for j in TTT[(coum * a - t_coum):(coum * a)]: 
            pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
        r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
    r1 = np.array(r1)  # 将 r1 转换为 numpy 数组
    mse = np.mean(np.square(r1))
    rmse = math.sqrt(mse)
    del r1, pp,
    return alp, mse, rmse

In [46]:
# 单进程 分函数
def calculate_parameters(linename, a, coum, TTT, data2, dict_columns_split, lin, NNN):
    PCC = 0
    lin = np.linspace(1,3,401)
    for t_coum in range(2, coum+1, 1):   
        # 求PCC
        alpha, pcc, temp_NNN, temp_TTT, pre, ori = alpha_pcc(t_coum, a, lin, linename, dict_columns_split, TTT, coum, data2, NNN)
        # 求MSE
        alp, mse, rmse = alp_mse_rmse(alpha, t_coum, a, linename, dict_columns_split, TTT, coum)
        if PCC == 0:
            Name_coum, Name_mse, Name_pcc, Name_alp, PCC, Name_rmse, Name_NNN, Name_TTT, Name_pre, Name_ori = t_coum, mse, pcc, alp, pcc, rmse, temp_NNN, temp_TTT, pre, ori
        if pcc > PCC:
            Name_coum, Name_mse, Name_pcc, Name_alp, PCC, Name_rmse, Name_NNN, Name_TTT, Name_pre, Name_ori = t_coum, mse, pcc, alp, pcc, rmse, temp_NNN, temp_TTT, pre, ori
        # print("name:{},coum:{:0d},alp={},mse={},pcc={},rmse={}".format(linename, t_coum, alp, mse, pcc,rmse),end="")
    del t_coum, mse, pcc, alp, rmse, temp_NNN, temp_TTT
    return [linename, Name_coum, Name_alp, Name_mse, Name_pcc, Name_rmse, Name_NNN, Name_TTT, Name_pre, Name_ori]

In [47]:
# 多进程
@timeit
def run_parallel(columns, a, lin, NNN):
    with multiprocessing.Pool() as pool:
        tp = []
        for idx, column in enumerate(columns):
            tp.append((column, a+idx, coum, TTT, data2, dict_columns_split, lin, NNN))
        results = pool.starmap(calculate_parameters, tp)
    # results = [calculate_result(i[0], i[1]) for i in results]
    return results

a = 1
results = run_parallel(columns, a, lin, NNN)    

run_parallel took 3023.613314 seconds


In [48]:
# 将结果保存到 CSV 文件中
pd.DataFrame(results).to_csv('pcc_cpu_test_multi.csv', header=None, index=False)
rebuild_list = []
for i in results:
    for j,k in zip(i[6],i[7]):
        rebuild_list.append([j,i[0],k,i[1],i[2],i[4],i[5]])
pd.DataFrame(rebuild_list,columns=["target_neighbor","target","similarity","neighbors","alp","pcc","rmse"]).to_csv("pcc_cpu_test_multi_target.csv")
temp_list = []
for i in results:
    for j,k,m in zip(i[8],i[9],cells_name):
        temp_list.append([i[0],m,j,k])
pd.DataFrame(temp_list,columns=["drugs","cells_line","pridict","origin"]).sort_values("cells_line").to_csv("pcc_cpu_test_multi_cells_line.csv")

In [49]:
ttpp = []
for i in results:
    ttpp.append(i[4])
print(sum(ttpp)/len(ttpp))

0.674870576675524


In [ ]:
# # 多线程
# @timeit
# def run_parallel(columns, a):
#     threads = []
#     a = 1
#     for column in columns:
#         thread = threading.Thread(target=calculate_parameters, args=(column, a, coum, TTT, data2, dict_columns_split))
#         threads.append(thread)
#         thread.start()
#         a += 1
    
#     # 等待所有线程完成
#     for thread in threads:
#         thread.join()

# # 加载保存的数组
# lin = np.load("lin.npy")
# run_parallel(columns, a)    

# # 将结果保存到 CSV 文件中
# pd.DataFrame(results).to_csv('pcc_cpu_test_multi.csv', header=None, index=False)

In [ ]:
# def calculate_result(linename, Name_coum):
#     t_coum = Name_coum
#     # 求PCC
#     alpha = []
#     result_pcc = []
#     pcc_split = []
#     for alp in lin:
#         pre = []
#         for elem in range(39):
#             pp = [] 
#             for j in TTT[(coum * a - t_coum):(coum * a)]: 
#                 pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))
#             pp = np.array(pp)  # 将 pp 转换为 numpy 数组
#             pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
#         pre = np.array(pre)  # 将 pre 转换为 numpy 数组
#         pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
#     result_pcc.append(np.max(pcc_split))
#     alpha.append(lin[pcc_split.index(np.max(pcc_split))])
#     pcc=result_pcc[0]
#     # 求MSE
#     r1 = []  # 初始化 r1 为空列表
#     alp = alpha[0]
#     for elem in range(39):
#         pp = []
#         for j in TTT[(coum * a - t_coum):(coum * a)]: 
#             pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  
#         r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) - dict_columns_split[f"{linename},{elem}"]['y_line'])
#     r1 = np.array(r1)  # 将 r1 转换为 numpy 数组
#     mse = np.mean(np.square(r1))
#     rmse = math.sqrt(mse)
#     return [linename, Name_coum, alp, mse, pcc, rmse]

# @timeit
# def run_parallel(columns, a):
#     with multiprocessing.Pool(processes=2) as pool:
#         results = []
#         for i, column in enumerate(columns):
#             result = pool.apply(calculate_parameters, (column, a + i, coum))
#             results.append(result)
#     return results